# The Sock Drawer 
*Problem 1, Fifty chanllenging problems in probability*

A drawer contains red socks and blacks socks. When two socks are drawn at random, the probability that both are red is $\frac{1}{2}$.
- How small can the number of socks in the number of socks in the drawer be?
- How small if the number of black socks is even?


# Mathematics

Let $n$, $k$ be the number of total socks and red socks respectively. Then we have
$$\frac{k}{n}\frac{k-1}{n-1}=\frac{1}{2}$$
which is equivalent to the algebraic equation
$$(2n-1)^{2}-2(2k-1)^{2}=-1.$$
Let $x=2n-1$, $y=2k-1$, we get the *Pell's equation*
$$x^{2}-2y^{2}=-1.$$
To solve the problem, it's not hard to find
- $n=4$, $k=3$ solves the first question
- $n=21$, $k=15$ solves the second question

However, the mathematics behind is more interesting---we're considering units ring $\mathcal{O}_{K}^{\times}$ in the number field $K=\mathbf{Q}[\sqrt{2}]$.


---


**Theorem**, *Fermat's dream and class field theory, 4.2*

Let $N\in \mathbf{Z}_{+}$ be a non-square integer. Let $P_{N}=\{(x,y)\in \mathbf{Z}\times\mathbf{Z}|x^{2}-Ny^{2}=\pm 1\}$ and $P_{N}'=\{(x,y)\in P_{N}|x\geq 1, y\geq 1\}$. Then
- There's a bijection between the units group $\mathbf{Z}[\sqrt{N}]^{\times}$ in $\mathbf{Z}[\sqrt{N}]$ and $P_{N}$:
$$\begin{align*}\theta:\mathbf{Z}[\sqrt{N}]^{\times}&\rightarrow P_{N}\\
x+y\sqrt{N}&\mapsto (x, y)
\end{align*}$$ 
- Let $(x_{0}, y_{0})\in P_{N}$ be a postive solution with minimal $x$-component, then it's also the solution with the minimal $y$-component, moreover
  - $\mathbf{Z}[\sqrt{N}]^{\times}=\{\pm(x_{0}+y_{0}\sqrt{N})^{n}|n\in \mathbf{Z}\}$
  - $\theta^{-1}(P_{N}')=\{(x_{0}+y_{0}\sqrt{N})^{n}|n\geq 1\}$
---

In our case, we have $N=2$, the units group $\mathbf{Z}[\sqrt{2}]$ is given by $\{\pm(1+\sqrt{2})^{n}|n\in \mathbf{Z}\}$. **In principle**, we can compute as many solutions as we want. For example, the first few positive solutions are given by coefficients of $(1+\sqrt{2})^{n}$, $n\geq 1$, namely
- $(1,1)$
- $(3,2)$
- $(7,5)$, this gives us the solution $n=4$, $k=3$
- $\dots$

The problem is that $P_{N}'$ contains more solutions than we need, we need the RHS of the equation to be $-1$, for the socks drawer problem, we also need $x$, $y$ to be odd integers.

# Recursive formula for continued fractions
---
**Theorem**

Given a continued fraction with terms $[a_{1}; a_{2}, \dots, a_{n}]$, the $i$-th numerator $p_{i}$ and the $i$-th denominator $q_{i}$ of the $i$-th convergent are defined by the recursive formula:
$$\begin{align*}p_{i}&=a_{i}p_{i-1}+p_{i-2}\\
q_{i}&= a_{i}q_{i-1}+q_{i-2}\end{align*}$$
where
$$\begin{align*}(p_{-1}, q_{-1})&= (0,1)\\
(p_{0}, q_{0})&= (1,0)\end{align*}$$
---
*Proof*: Induction on $n$. The first few cases can be checked directly. Let $C_{k}=[a_{1}; a_{2}, \dots, a_{k}]$, then $C_{k+1}=[a_{1}; a_{2}, \dots, a_{k+1}]$ can be viewed as $[a_{1}; a_{2}, \dots, a_{k}+\frac{1}{a_{k+1}}]$, then use the induction on $k$, a direct computation proves the recursive formula.



# Coding
*Solving Pell's equations with Python*

## Step 2: Dynamic programming
Let $a_{n}+b_{n}\sqrt{N}$ be the $n$-th solution, then we have 
$$\begin{align*}a_{n+1}+b_{n+1}\sqrt{N}&=(a_{n}+b_{n}\sqrt{N})(a_{0}+b_{0}\sqrt{N})\\
&=(a_{0}a_{n}+Nb_{0}b_{n})+(b_{0}a_{n}+a_{0}b_{n})\sqrt{N}\end{align*}.$$
Namely, we have the recursive formula
$$\begin{align*}a_{n+1}&=a_{0}a_{n}+Nb_{0}b_{n}\\
b_{n+1}&=b_{0}a_{n}+a_{0}b_{n}\end{align*}$$

## Step 1: Use continued fractions to find the fundamental solution $a_{0}+b_{0}\sqrt{N}$.

In [0]:
from math import sqrt
def fundamental_solution(N, rhs = -1, num_try = 1000):
  """Try to find the fundamental solution to Pell's equation x^2-Ny^2=rhs within num_try rounds.
     Note that because of the precision of float numbers in the computer system, the computation for convergents of sqrt{N} might be wrong
     but we can always check the final answer to make sure we do get the correct answer.

  :param: N, int
  :param: rhs, int
  :param: num_try, int
  """
  assert N>=2 and int(sqrt(N))**2 != N, "Expected N to be a positive square-free integer, get {} instead".format(N)
  
  current_integral = int(sqrt(N))
  current_res = sqrt(N)-current_integral

  p_older, q_older = 0,1
  p_old, q_old = 1,0

  for k in range(num_try):
    #print(current_integral)
    p = current_integral*p_old+p_older
    q =  current_integral*q_old+q_older
    if p**2-N*(q**2)==rhs:
      return [p,q]
    else:
      p_older, q_older = p_old, q_old
      p_old, q_old = p, q
      current_integral = int(1/current_res)
      current_res = 1/current_res-current_integral
  return "Fundamental solution not found in {} rounds".format(num_try)



def pell_recursion(fundamental, N, rhs=-1, step=10):
  """Given the fundamental solution to x^2-Ny^2=-1, compute the first n(=step) solutions

  """
  ans = [fundamental]
  current_x = fundamental[0]
  current_y = fundamental[1]
  while(len(ans)<step):
    x = fundamental[0]*current_x+N*fundamental[1]*current_y
    y = fundamental[1]*current_x+fundamental[0]*current_y
    current_x = x
    current_y = y
    if x**2-N*(y**2)==rhs:
      ans.append([x, y])
  return ans

def pell(N, step = 0, rhs = -1):
  """return the n(=step)-th positive solution of x^2-Ny^2=rhs, rhs = -1 by default.

  :param: N, int
  :param: step, int
  :param: rhs, int

  """
  pass

In [34]:
fundamental_solution(61, rhs=-1, num_try=100000)

[29718, 3805]

In [4]:
ans = pell_recursion([1,1], 2)
for ele in ans:
  print("{}^2-2*{}^2={}".format(ele[0], ele[1],ele[0]**2-2*(ele[1]**2)))

1^2-2*1^2=-1
7^2-2*5^2=-1
41^2-2*29^2=-1
239^2-2*169^2=-1
1393^2-2*985^2=-1
8119^2-2*5741^2=-1
47321^2-2*33461^2=-1
275807^2-2*195025^2=-1
1607521^2-2*1136689^2=-1
9369319^2-2*6625109^2=-1


# Don't reinvent the wheels
*Solving Pell's equation with SageMath, ```sympy.solvers.diaphantine``` or any online solvers*
- [```sympy.solvers.diaphantine```](https://https://docs.sympy.org/latest/modules/solvers/diophantine.html)

In [35]:
from sympy.solvers.diophantine import diophantine, diop_DN
diop_DN(61, -1)

[(29718, 3805)]

# Remarks

The naive algorithm that we use is not very efficient, it works if the fundamental solution is not so big. 